In [2]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog, boxscoretraditionalv2
import pandas as pd
import time
from requests.exceptions import ConnectionError

# 抓出NBA球隊
nba_teams = teams.get_teams()

# 輸入球隊名稱
team_name = input("請輸入 NBA 球隊名稱：")

# 檢查球隊是否存在
team_list = [team for team in nba_teams if team['full_name'] == team_name]
if not team_list:
    print("找不到該球隊，請確認名稱是否正確。")
else:
    team_id = team_list[0]['id']

    # 抓取每場比賽
    try:
        gamelog = teamgamelog.TeamGameLog(team_id=team_id, season='2024-25', season_type_all_star='Regular Season')
        df_games = gamelog.get_data_frames()[0]
        game_ids = df_games['Game_ID'].tolist()
    except ConnectionError:
        print("無法連線到 NBA API，請檢查網路或稍後再試。")
        exit()

    team_players ={
            'Golden State Warriors': ["Stephen Curry", "Jimmy Butler III", "Buddy Hield", "Draymond Green", "Jonathan Kuminga"],
            'Los Angeles Lakers': ["LeBron James", "Austin Reaves", "Luka Dončić", "Rui Hachimura", "Jaxson Hayes"],
            'Denver Nuggets': ["Nikola Jokić", "Michael Porter Jr.", "Jamal Murray", "Christian Braun", "Aaron Gordon"],
            'Los Angeles Clippers': ["James Harden", "Kawhi Leonard", "Norman Powell", "Ivica Zubac", "Bogdan Bogdanović"],
            'Phoenix Suns': ["Devin Booker", "Kevin Durant", "Bradley Beal", "Grayson Allen", "Mason Plumlee"],
            'Sacramento Kings': ["DeMar DeRozan", "Domantas Sabonis", "Keegan Murray", "Zach LaVine", "Malik Monk"],
            'Minnesota Timberwolves': ["Anthony Edwards", "Rudy Gobert", "Julius Randle", "Jaden McDaniels", "Donte DiVincenzo"],
            'Oklahoma City Thunder': ["Shai Gilgeous-Alexander", "Jalen Williams", "Chet Holmgren", "Isaiah Hartenstein", "Luguentz Dort"],
            'Portland Trail Blazers': ["Scoot Henderson", "Anfernee Simons", "Deandre Ayton", "Deni Avdija", "Jerami Grant"],
            'Utah Jazz': ["Lauri Markkanen", "Walker Kessler", "John Collins", "Isaiah Collier", "Collin Sexton"],
            'Dallas Mavericks': ["Anthony Davis", "Klay Thompson", "Dereck Lively II", "Max Christie", "Kyrie Irving"],
            'Houston Rockets': ["Amen Thompson", "Jalen Green", "Fred VanVleet", "Dillon Brooks", "Alperen Sengun"],
            'Memphis Grizzlies': ["Ja Morant", "Desmond Bane", "Jaren Jackson Jr.", "Zach Edey", "Scotty Pippen Jr."],
            'New Orleans Pelicans': ["Dejounte Murray", "CJ McCollum", "Trey Murphy III", "Herbert Jones", "Zion Williamson"],
            'San Antonio Spurs': ["Victor Wembanyama", "De'Aaron Fox", "Stephon Castle", "Jeremy Sochan", "Harrison Barnes"],
            'New York Knicks': ["Jalen Brunson", "Mikal Bridges", "Josh Hart", "OG Anunoby", "Karl-Anthony Towns"],
            'Boston Celtics': ["Jrue Holiday", "Derrick White", "Jaylen Brown", "Jayson Tatum", "Kristaps Porziņģis"],
            'Brooklyn Nets': ["D'Angelo Russell", "Nic Claxton", "Cameron Johnson", "Ziaire Williams", "Keon Johnson"],
            'Philadelphia 76ers': ["Joel Embiid", "Kelly Oubre Jr.", "Tyrese Maxey", "Guerschon Yabusele", "Paul George"],
            'Toronto Raptors': ["RJ Barrett", "Jakob Poeltl", "Ochai Agbaji", "Scottie Barnes", "Gradey Dick"],
            'Chicago Bulls': ["Nikola Vučević", "Kevin Huerter", "Coby White", "Josh Giddey", "Matas Buzelis"],
            'Cleveland Cavaliers': ["Donovan Mitchell", "Darius Garland", "Jarrett Allen", "Max Strus", "Evan Mobley"],
            'Detroit Pistons': ["Cade Cunningham", "Tim Hardaway Jr.", "Tobias Harris", "Ausar Thompson", "Jalen Duren"],
            'Indiana Pacers': ["Myles Turner", "Pascal Siakam", "Tyrese Haliburton", "Andrew Nembhard", "Aaron Nesmith"],
            'Milwaukee Bucks': ["Giannis Antetokounmpo", "Brook Lopez", "Damian Lillard", "Taurean Prince", "Kyle Kuzma"],
            'Atlanta Hawks': ["Clint Capela", "Trae Young", "Jalen Johnson", "Dyson Daniels", "Zaccharie Risacher"],
            'Charlotte Hornets': ["Miles Bridges", "LaMelo Ball", "Mark Williams", "Josh Green", "Brandon Miller"],
            'Miami Heat': ["Andrew Wiggins", "Bam Adebayo", "Tyler Herro", "Davion Mitchell", "Kel'el Ware"],
            'Orlando Magic': ["Kentavious Caldwell-Pope", "Jalen Suggs", "Franz Wagner", "Paolo Banchero", "Wendell Carter Jr."],
            'Washington Wizards': ["Jordan Poole", "Alex Sarr", "Kyshawn George", "Corey Kispert", "Bub Carrington"],
    }

    players = team_players.get(team_name)
    player_plus_minus = {player: [] for player in players}

    # 抓取每場比賽中的正負值
    for game_id in game_ids:
        try:
            boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
            df_boxscore = boxscore.get_data_frames()[0]
            
            for player in players:
                player_row = df_boxscore[df_boxscore['PLAYER_NAME'] == player]
                if not player_row.empty:
                    plus_minus = player_row.iloc[0]['PLUS_MINUS']
                    player_plus_minus[player].append(plus_minus)
                    
            time.sleep(1)
        except ConnectionError:
            print(f"無法取得比賽 {game_id} 數據，跳過此比賽。")

    # 計算並輸出平均正負值
    data = []
    for player in players:
        series = pd.Series(player_plus_minus[player])
        if not series.empty:
            avg = series.mean()
        else:
            avg = None
        data.append({"球隊": team_name, "球員": player, "平均正負值": avg})

    df_result = pd.DataFrame(data)

    # 輸出結果
    print(df_result)

    # 存成 CSV檔
    file_name = f"{team_name.replace(' ', '_')}_plus_minus.csv"
    df_result.to_csv(file_name, index=False, encoding='utf-8')
    print(f"已儲存為 {file_name}")

請輸入 NBA 球隊名稱： 123


找不到該球隊，請確認名稱是否正確。
